# 线程池

## 什么是线程池
- 线程池是存放多个线程的容器
- CPU调度线程执行后不会销毁线程
- 将线程返回线程池重复利用
## 为什么需要线程池
- 线程是稀缺资源，不应该频繁创建和销毁，创建和销毁需要额外资源
- 架构解耦，线程创建和业务处理解耦，更加优雅
- 线程池是使用线程的最佳实践
![](images/2022-08-28-22-12-29.png)
解耦就是努力使程序积木化
我们应该使用线程池把所有需要的线程先创立起来，这样可以实现架构的解耦

## 线程池的实现

### 任务处理线程ProcessThread
- 任务处理线程需要不断地从任务队列里取任务执行
- 任务处理线程需要有一个标记，标记线程什么时候应该停止
  ![](images/2022-08-28-22-16-35.png)

In [4]:
import threading
class ProcessThread(threading.Thread):


    def __init__(self,task_queue,*args,**kwargs):
        threading.Thread.__init__(self,*args,**kwargs)
        #停止标记
        self.dismiss_flag=threading.Event()
        #任务队列
        self.task_queue=task_queue
        self.args=args
        self.kwargs=kwargs
        
    def run(self):
        while True:
            #判断线程是否被要求停止
            if self.dismiss_flag.is_set():
                break
            task=self.task_queue.pop()
            if not isinstance(task,Task):
                #忽略这次任务
                continue
            # 执行task实际逻辑（是通过函数调用引进来的）
            result=task.callable(*task.args,**task.kwargs)

    def dismiss(self):
        self.dismiss_flag.set()

    def stop(self):
        self.dismiss()

### 实现任务处理线程池Pool
1. 存放多个任务处理线程
2. 负责多个线程的启停
3. 管理向线程池的提交任务，下发给线程去执行 
![](images/2022-08-29-00-17-17.png)

In [5]:
from concurrent.futures import thread
import psutil
import threading

class TaskTypeErrorException:
    pass


class ThreadPool:

    def __init__(self,size=0):
        if not size:
            #约定线程池的大小为CPU核数的两倍（最佳实践）
            size=psutil.cpu_count()*2
        #线程池
        self.pool=ThreadSafeQueue(size)
        #任务队列
        self.task_queue=ThreadSafeQueue()
        for i in range(size):
            self.pool.put(ProcessThread(self.task_queue))
    def start(self):
        for i in range(self.pool.size()):
            thread=self.pool.get(i)
            thread.start()
        

            
    def join(self):
        for i in range(self.pool.size()):
            thread=self.pool.get(i)
            thread.stop()
        while self.pool.size:
            thread=self.pool.pop()
            thread.join()

    def put(self,item):
        if not isinstance(item,Task):
            raise TaskTypeErrorException()
        self.put(item)

    def batch_put(self,item_list):
        if not isinstance(item_list,list):
            item_list=list(item_list)
        for item in item_list:
            self.put(item)
        
    def size(self):
        return self.pool.size()